# Introduction
Creating a large assembly from scratch is time consuming, therefore, a selection on classes have been provided within BOM analysis to parse a skeleton which can be loaded into an assembly.

## Tokamak Parts
Parts contains the parts which will build the skeleton. Every part has a type and a reference, the type is the key in the part dictionary. The reference (called by .ref) is analogous to a part number which should be unique to that part. The parts can be split across multiple files or dictionaries that will be read into a single skeleton.

In [ ]:
parts = {
    "assembly": {
        "class_str": ["bom_analysis.bom.Assembly"],
        "params_name": ["assembly"],
    },
    "component": {
        "class_str": ["bom_analysis.bom.Component"],
        "params_name": ["component"],
    },
}
tokamak_parts = {
    "blanket": {
        "inherits": ["assembly"],
        "children": {
            "breeding_zone": {"type": "breeding_pins"},
            "manifold": {"type": "double_wall_mf"},
        },
    },
    "breeding_pins": {
        "inherits": ["component"],
        "test_list": ["hello"],
        "params_name": ["pins"],
    },
    "double_wall_mf": {"inherits": ["component"], "material": {"name": "steel"}},
}
extra_info = {
    "double_wall_mf": {"description": "a manifold with a double wall"},
    "breeding_pins": {"test_list": ["world"]},
}

The dictionaries can be merged together using normal methods or with the UpdateDict class in framework.utils. UpdateDict aims to allow nested updates of dictionaries, lists etc.

In [ ]:
from bom_analysis.utils import UpdateDict
import pprint

UpdateDict(tokamak_parts, parts, extra_info)
pprint.pprint(tokamak_parts)

Note how the lists are merged.

## Parsing
The dictionaries (defined or loaded in from files) can then be parsed together to form a full skeleton based on the above imporant keys

There are some important strings in these dictionaries.

* "inherits" - the strings in this list will be searched for within any part dictionaries and the dictionary from the item that is inherited from will be merged
* "class_str" - string within class_str list will be used to create the classes from strings. In-built can be passed.
* "children" - defines lower level in the hierarchy with the keys in the dict becoming and the "type" being used to search the parts
* "params_name" - it is optional but a set of parameters can also be loaded into the skeleton by specifying a key to their dictionary in params_name

The created skeleton is written to the skeleton attribute of the SkeletonParser instance. For this example, each stage of parsing related to the above strings will be taken individually. In the end, a method will be given that handles everything for the user.

To create a skeleton, the type and reference of the component at the top of the hierarchy are required.

In [ ]:
from bom_analysis.parsers import SkeletonParser

parsed = SkeletonParser()

parsed.spine(
    component_ref="my_new_blanket",
    component_type="blanket",
    component_database=tokamak_parts,
)

pprint.pprint(parsed.skeleton)

Only the required components are now shown in the skeleton with the references acting as keys. Some metadata has been added on the top reference and type to aid with loading in the future.

The components can then be inherited and the parameters added. For the parameters, dictionary of is needed with the information.

In [ ]:
parameters = {
    "component": {
        "mass": {"var": "mass", "description": "the mass of the component"},
        "temperature": {
            "var": "temperature",
            "description": "the component temperature",
        },
    },
    "assembly": {
        "envelope_volume": {
            "var": "envelope_volume",
            "description": "the volumetric envelope occupied by the assembly",
        }
    },
    "pins": {"diameter": {"var": "diameter", "description": "the diameter of the pin"}},
}

parsed.add_bones(tokamak_parts, parameters)
pprint.pprint(parsed.skeleton)

From this it can be seen that the params_name and inherits have been removed and replaced with _params and inherited.

This process can be replicated again with the component dictionary method.

In [ ]:
new_parsed = SkeletonParser()
new_parsed.component_dictionary(
    component_ref="my_second_blanket",
    component_type="blanket",
    component_database=tokamak_parts,
    parameter_dictionary=parameters,
)
pprint.pprint(new_parsed.skeleton)

These can then easily be loaded into an assembly.

In [ ]:
from bom_analysis import Assembly

first_blanket = Assembly()
first_blanket.from_dict(parsed.skeleton)
second_blanket = Assembly()
second_blanket.from_dict(new_parsed.skeleton)
print(first_blanket.params)
print(first_blanket.params.envelope_volume)
print(second_blanket.manifold.material)

## Conclusion
There are two extra examples on the parsing of skeletons, one which uses the Config to repeat what has been done in this example and one which updates the skeleton using a settings file